## Summary Statistics Tables

In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels import AbsorbingLS
from linearmodels import PanelOLS
import itertools
import warnings
from linearmodels.iv.absorbing import AbsorbingEffectWarning
import psutil
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [28]:
# Get the virtual memory details
memory_info = psutil.virtual_memory()
total_memory = memory_info.total
available_memory = memory_info.available
used_memory = memory_info.used

print(f"Total Memory: {total_memory / (1024 ** 3):.2f} GB")
print(f"Available Memory: {available_memory / (1024 ** 3):.2f} GB")
print(f"Used Memory: {used_memory / (1024 ** 3):.2f} GB")

Total Memory: 7.45 GB
Available Memory: 0.67 GB
Used Memory: 6.78 GB


In [29]:
df = pd.read_csv("./Final Data/analysis_input.csv")
df['principal_city_binary'] = df['principal_city'] = np.where(df['principal_city'] == 999999, np.nan, np.where(df['principal_city'] == 0, 0, 1))
df['urban_rural_binary'] = np.where(df['urbanrural'].isin(['M', 'U']), 1, np.where(df['urbanrural'] == 'R', 0, np.nan))

## Table 1 Approval List

#### Manufacturers and Traders Trust Company  Hudson City Savings Bank	 2015
#### PNC Bank, National Association RBC Bank (USA)	2011
#### The Huntington National Bank Firstmerit Bank, National Association 2016
#### Fifth Third Bank MB Financial Bank 2018
#### Branch Banking and Trust Company Susquehanna Bank	2015
#### Harris National Association M&I Marshall and Ilsley Bank	2012
#### KeyBank National Association	First Niagara Bank, National Association  2016
#### TCF National Bank	Chemical Bank 2019
#### Sterling National Bank Astoria Bank 2017

## Table 2 Merger Summary Statistics

In [30]:
round(df.drop_duplicates(subset=['ACQ_CERT','mergerID'])[['ACQ_CERT','mergerID','premerger_acqbranches','premerger_acqassets','premerger_acqstates','premerger_acqcounties','premerger_acqtracts']].describe(),0)

,ACQ_CERT,mergerID,premerger_acqbranches,premerger_acqassets,premerger_acqstates,premerger_acqcounties,premerger_acqtracts
count,8.0,8.0,8.0,8.0,8.0,8.0,8.0
mean,11561.0,8.0,948.0,112457808.0,10.0,180.0,888.0
std,8792.0,4.0,612.0,75049206.0,4.0,132.0,576.0
min,588.0,0.0,209.0,23198143.0,3.0,32.0,184.0
25%,6516.0,6.0,578.0,64461326.0,7.0,98.0,537.0
50%,8259.0,8.0,832.0,91271354.0,10.0,164.0,784.0
75%,16812.0,11.0,1279.0,149930283.0,13.0,231.0,1221.0
max,28330.0,13.0,1860.0,251075292.0,16.0,434.0,1708.0


In [31]:
round(df.drop_duplicates(subset=['OUT_CERT','mergerID'])[['ACQ_CERT','mergerID','premerger_outbranches','premerger_outassets','premerger_outstates','premerger_outcounties','premerger_outtracts']].describe(), 0)

,ACQ_CERT,mergerID,premerger_outbranches,premerger_outassets,premerger_outstates,premerger_outcounties,premerger_outtracts
count,8.0,8.0,8.0,8.0,8.0,8.0,8.0
mean,11561.0,8.0,235.0,28970659.0,4.0,56.0,223.0
std,8792.0,4.0,100.0,10181403.0,2.0,32.0,96.0
min,588.0,0.0,84.0,18317709.0,2.0,6.0,74.0
25%,6516.0,6.0,196.0,20157092.0,3.0,36.0,188.0
50%,8259.0,8.0,232.0,25819592.0,4.0,60.0,220.0
75%,16812.0,11.0,274.0,38023931.0,5.0,73.0,258.0
max,28330.0,13.0,377.0,44975270.0,8.0,99.0,359.0


## Table 3 —Summary Statistics for Exposed and Control Tracts

In [32]:
df_sample = df.dropna(subset=['overlap'])
df_other = df[df['overlap'].isna()]

In [33]:
tract_comparison_list = ['vol_100k', 'vol_250k', 'vol_1mil', 'vol_sbus', 'sbl_total', 'sbl_total_num', 'num_sbus', 'num_100k', 'num_250k', 'num_1mil', 'medincome', 'popden', 'poptot', 'pminority', 'principal_city_binary',
                          'urban_rural_binary','pcollege','total_branches_tract','avg_branch_growth_tract']

In [34]:
df_sample['pcollege'].describe()

count    38396.000000
mean         0.643431
std          0.163853
min          0.000000
25%          0.527613
50%          0.649034
75%          0.773348
max          1.000000
Name: pcollege, dtype: float64

#### Exposed Tracts

In [35]:
exposed_dscp = df_sample[df_sample['overlap']==1].drop_duplicates(subset=['year','state','county','tract'])[tract_comparison_list].describe()
round(exposed_dscp,2).iloc[:,4:]

,sbl_total,sbl_total_num,num_sbus,num_100k,num_250k,num_1mil,medincome,popden,poptot,pminority,principal_city_binary,urban_rural_binary,pcollege,total_branches_tract,avg_branch_growth_tract
count,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1700.00,1421.00,1468.00,1700.00,1700.00,1593.00
mean,10987.32,182.68,73.69,160.11,10.72,11.85,52946.02,2973.77,4660.86,21.44,0.28,0.98,0.61,4.34,0.01
std,26609.28,303.83,104.03,249.99,27.53,31.85,21379.86,3812.23,1934.60,18.27,0.45,0.14,0.15,3.94,0.41
min,41.00,9.00,3.00,9.00,0.00,0.00,9523.00,52.17,963.00,1.44,0.00,0.00,0.17,1.00,-0.67
25%,2265.00,64.75,28.00,58.00,2.00,2.00,39632.00,990.79,3286.00,7.19,0.00,1.00,0.51,2.00,-0.17
50%,5189.00,115.00,48.00,102.00,5.00,5.00,51641.00,2018.88,4499.50,14.64,0.00,1.00,0.62,4.00,0.00
75%,11412.75,202.00,85.00,181.00,11.00,12.00,63750.00,3257.09,5899.00,31.62,1.00,1.00,0.71,5.00,0.00
max,566395.00,4583.00,1388.00,3213.00,661.00,709.00,130313.00,31545.45,12887.00,91.89,1.00,1.00,0.98,44.00,7.00


#### Other Tracts

In [36]:
other_dscp = df_other.drop_duplicates(subset=['year','state','county','tract'])[tract_comparison_list].describe()
round(other_dscp,2).iloc[:,4:]

,sbl_total,sbl_total_num,num_sbus,num_100k,num_250k,num_1mil,medincome,popden,poptot,pminority,principal_city_binary,urban_rural_binary,pcollege,total_branches_tract,avg_branch_growth_tract
count,71207.00,71207.00,71207.00,71207.00,71207.00,71207.00,71207.00,71202.00,71207.00,71207.00,58905.00,60323.00,71107.00,71207.00,61738.00
mean,5010.06,106.15,49.22,96.18,5.05,4.93,56851.88,3472.29,4599.56,26.47,0.33,0.94,0.58,1.88,0.06
std,8027.80,116.68,48.44,102.68,9.58,9.13,27614.75,5607.01,1995.02,23.91,0.47,0.23,0.17,1.44,0.36
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,-0.91
25%,1457.00,46.00,21.00,42.00,1.00,1.00,38101.00,645.59,3248.00,8.23,0.00,1.00,0.45,1.00,0.00
50%,3034.00,77.00,37.00,70.00,3.00,3.00,50718.00,2093.25,4361.00,18.26,0.00,1.00,0.57,2.00,0.00
75%,5894.00,129.00,62.00,117.00,6.00,6.00,69360.50,4104.46,5633.00,36.89,1.00,1.00,0.71,2.00,0.00
max,566395.00,4583.00,1777.00,3311.00,661.00,709.00,250001.00,161916.67,26230.00,100.00,1.00,1.00,1.00,44.00,7.00


#### Control Tracts

In [37]:
control_dscp = df_sample[df_sample['overlap']==0].drop_duplicates(subset=['year','state','county','tract'])[tract_comparison_list].describe()
round(control_dscp,2).iloc[:,4:]

,sbl_total,sbl_total_num,num_sbus,num_100k,num_250k,num_1mil,medincome,popden,poptot,pminority,principal_city_binary,urban_rural_binary,pcollege,total_branches_tract,avg_branch_growth_tract
count,24658.00,24658.00,24658.00,24658.00,24658.00,24658.00,24658.00,24658.00,24658.00,24658.00,20489.00,21290.00,24658.00,24658.00,22097.00
mean,10079.83,192.62,85.16,172.48,9.94,10.20,61347.14,3660.04,4893.25,27.44,0.39,1.00,0.65,4.32,0.04
std,15380.15,216.93,81.40,188.67,18.07,17.23,30474.55,6491.97,2345.54,20.82,0.49,0.07,0.16,2.46,0.35
min,19.00,2.00,1.00,2.00,0.00,0.00,0.00,10.91,9.00,0.00,0.00,0.00,0.00,1.00,-0.86
25%,3020.00,83.00,39.00,75.00,2.00,2.00,39875.00,1260.10,3427.25,11.62,0.00,1.00,0.53,3.00,0.00
50%,5926.00,137.00,65.00,124.00,5.00,5.00,55174.00,2380.62,4645.00,21.32,0.00,1.00,0.65,4.00,0.00
75%,11503.75,230.00,107.00,208.00,11.00,12.00,76509.00,4158.62,6010.75,37.93,1.00,1.00,0.78,5.00,0.00
max,566395.00,5897.00,1944.00,5152.00,661.00,709.00,250001.00,158769.23,38169.00,100.00,1.00,1.00,1.00,44.00,8.00


#### P-values on difference

In [38]:
#df = df.drop_duplicates(subset=['year','state','county','tract'])

In [39]:
# Dictionary to store results
exposed_other_p_values = {}
exposed_control_p_values = {}

df['exposed_other'] = np.where(df['overlap'] == 1, 1, np.where(df['overlap'] == 0, np.nan, 0))
df['exposed_control'] = np.where(df['overlap'] == 1, 1, np.where(df['overlap'] == 0, 0, np.nan))

df_1 = df.dropna(subset=['exposed_other'])
df_2 = df.dropna(subset=['exposed_control'])

# Run regressions and extract p-values for the "exposed" variable
for characteristic in tract_comparison_list:
    #print(characteristic)
    formula = f'{characteristic} ~ exposed_other '
    model = smf.ols(formula=formula, data=df_1).fit()
    exposed_other_p_values[characteristic] = model.pvalues['exposed_other']

# Run regressions and extract p-values for the "exposed" variable
for characteristic in tract_comparison_list:
    #print(characteristic)
    formula = f'{characteristic} ~ exposed_control'
    model = smf.ols(formula=formula, data=df_2).fit()
    exposed_control_p_values[characteristic] = model.pvalues['exposed_control']

# Display the p-values
exposed_other_df = round(pd.DataFrame.from_dict(exposed_other_p_values, orient='index', columns=['p_value']), 3)
# Display the p-values
exposed_control_df = round(pd.DataFrame.from_dict(exposed_control_p_values, orient='index', columns=['p_value']), 3)

In [40]:
exposed_other_df

,p_value
vol_100k,0.000
vol_250k,0.000
vol_1mil,0.000
vol_sbus,0.000
sbl_total,0.000
sbl_total_num,0.000
num_sbus,0.000
num_100k,0.000
num_250k,0.000
num_1mil,0.000


In [41]:
exposed_control_df

,p_value
vol_100k,0.001
vol_250k,0.000
vol_1mil,0.000
vol_sbus,0.014
sbl_total,0.000
sbl_total_num,0.000
num_sbus,0.581
num_100k,0.000
num_250k,0.000
num_1mil,0.000


## Table 4—Representativeness of the Merger Sample

In [42]:
sod_df = pd.read_csv("./Wrangled Data/sod_labelled_99_22.csv")
cra_df = pd.read_csv("./Wrangled Data/cra_wrangled_all.csv")
controls_df = pd.read_csv("./Wrangled Data/controls_all.csv")
# Renaming the columns for consistent merging
cra_df_renamed = cra_df.rename(columns={'activity_year': 'year', 'census_tract': 'tract'})
controls_df_renamed = controls_df.rename(columns={'Date': 'year', 'census_tract': 'tract'})
sod_df_renamed = sod_df.rename(columns={'YEAR': 'year', 'State': 'state', 'County': 'county','Tract': 'tract'})

# Merging the dataframes
merged_df = cra_df_renamed.merge(controls_df_renamed, on=['year', 'state', 'county', 'tract'], how='inner')
merged_df = merged_df.merge(sod_df_renamed, on=['year', 'state', 'county', 'tract'], how='inner')

merged_df['principal_city_binary'] = merged_df['principal_city'] = np.where(merged_df['principal_city'] == 999999, np.nan, np.where(merged_df['principal_city'] == 0, 0, 1))
merged_df['urban_rural_binary'] = np.where(merged_df['urbanrural'].isin(['M', 'U']), 1, np.where(merged_df['urbanrural'] == 'R', 0, np.nan))

C:\Users\yulon\AppData\Local\Temp\ipykernel_27184\2898482551.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  controls_df = pd.read_csv("./Wrangled Data/controls_all.csv")


In [43]:
def count_branch_and_closure(sod_df, level_indicator, merge_df):
    sod_df_copy = sod_df.copy()
    merge_copy = merge_df.copy()

    # Step 5.3: Calculate average branch growth over the previous two years
    def calculate_avg_branch_growth(group):
        # Calculate the difference in total branches between the current year and two years ago
        return (group - group.shift(2)) / group.shift(2)  # Percentage
    
    if level_indicator == 'tract':
        group_lst1 = ['year','state','county','tract']
        group_lst2 = ['state','county','tract'] ; level = 'level_3'
        var_1 = 'total_branches_tract'; var_2 = 'branch_closures_tract'; var_3 = 'avg_branch_growth_tract'

    elif level_indicator == 'county':
        group_lst1 = ['year','state','county']
        group_lst2 = ['state','county'] ; level = 'level_2'
        var_1 = 'total_branches_county'; var_2 = 'branch_closures_county'; var_3 = 'avg_branch_growth_county'

    elif level_indicator == 'merger':
        group_lst1 = ['mergerID','year','state','county','tract']
        group_lst2 = ['mergerID','state','county','tract'] ; level = 'level_4'
        var_1 = 'total_branches'; var_2 = 'branch_closures'; var_3 = 'avg_branch_growth_merger'

    # Use 2010-2022 data and sort the data frame
    sod_df_10_22 = sod_df_copy[sod_df_copy['year']>= 2010].sort_values(group_lst1)

    # Count the number of branches in each tract
    sod_df_10_22[var_1] = sod_df_10_22.groupby(group_lst1)['CERT'].transform('count')

    # Calculate yearly number of closures in each tract, negate the result for correct economic meaning
    clo = 0 - sod_df_10_22.drop_duplicates(subset=group_lst1).groupby(group_lst2)[var_1].diff(1)

    # save indecies for correct matching
    index = clo.index

    # Use the correct indices
    sod_correc = sod_df_10_22.loc[index]

    # Assign the result
    sod_correc[var_2] = clo
    
    # Calculate Average growth of branches
    temp_df = sod_correc.groupby(group_lst2)[var_1].apply(calculate_avg_branch_growth).reset_index()
    #print(temp_df)
    
    # Calculate yearly number of closures in each tract, negate the result for correct economic meaning
    growth = temp_df[var_1]
        
    # save indecies for correct matching
    index_growth = temp_df[level]
        
    # Use the correct indices
    #correc_df = sod_df_10_22.loc[index]
    growth.index = index_growth

    # Assign the result
    sod_correc[var_3] = growth
        
    # Merge
    
    merge_copy = merge_copy.merge(sod_correc[group_lst1 + [var_1] + [var_2] + [var_3]],  on = group_lst1, how = 'left')

    # Fill only the NaN's due to duplication 
    merge_copy[var_2] = merge_copy.groupby(group_lst2)[var_2].ffill()
    merge_copy[var_3] = merge_copy.groupby(group_lst2)[var_3].ffill()


    return merge_copy

In [44]:
## drop duplicates because the purpose here is summary statistics instead of merger-based analysis
sod_all_tract_no_dup = count_branch_and_closure(sod_df_renamed, 'tract', merged_df).drop_duplicates(subset = ['year','state','county','tract'])

In [45]:
branched_tracts = sod_all_tract_no_dup[sod_all_tract_no_dup['total_branches_tract']>0]
with_closings_tracts = branched_tracts[branched_tracts['branch_closures_tract']>0]
merger_sample = df.drop_duplicates(subset=['year', 'state', 'county', 'tract']).copy()

In [46]:
round(branched_tracts[tract_comparison_list].describe(), 2)

,vol_100k,vol_250k,vol_1mil,vol_sbus,sbl_total,sbl_total_num,num_sbus,num_100k,num_250k,num_1mil,medincome,popden,poptot,pminority,principal_city_binary,urban_rural_binary,pcollege,total_branches_tract,avg_branch_growth_tract
count,294628.00,294628.00,294628.00,294628.00,294628.00,294628.00,294628.00,294628.00,294628.00,294628.00,294628.00,294593.00,294628.00,294628.00,245668.00,251932.00,294245.00,294628.00,245323.00
mean,1784.19,905.23,2706.47,1740.05,5395.89,124.86,57.79,114.48,5.31,5.06,58307.38,5333.48,4686.10,33.83,0.41,0.95,0.60,1.91,0.04
std,2357.39,1652.21,5111.56,2042.54,8650.20,141.69,58.68,127.21,9.87,9.39,29432.76,11834.28,2095.08,26.83,0.49,0.21,0.18,1.53,0.33
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,-0.91
25%,631.00,156.00,379.00,528.00,1480.00,52.00,24.00,47.00,1.00,1.00,38079.00,869.27,3304.00,12.09,0.00,1.00,0.46,1.00,0.00
50%,1169.00,475.00,1307.00,1157.00,3109.00,89.00,42.00,82.00,3.00,3.00,51667.00,2577.59,4446.00,25.65,0.00,1.00,0.59,1.00,0.00
75%,2129.00,1070.00,3126.00,2232.00,6205.00,151.00,73.00,139.00,6.00,6.00,71798.00,5148.34,5761.25,49.74,1.00,1.00,0.73,2.00,0.00
max,107825.00,110028.00,370254.00,75320.00,566395.00,5897.00,2140.00,5152.00,661.00,709.00,250001.00,220000.00,53812.00,100.00,1.00,1.00,1.00,44.00,8.00


In [47]:
round(with_closings_tracts[tract_comparison_list].describe(), 2)

,vol_100k,vol_250k,vol_1mil,vol_sbus,sbl_total,sbl_total_num,num_sbus,num_100k,num_250k,num_1mil,medincome,popden,poptot,pminority,principal_city_binary,urban_rural_binary,pcollege,total_branches_tract,avg_branch_growth_tract
count,17937.00,17937.00,17937.0,17937.00,17937.00,17937.00,17937.00,17937.00,17937.00,17937.00,17937.00,17935.00,17937.00,17937.00,15879.00,15350.00,17915.00,17937.00,15967.00
mean,2677.57,1419.90,4349.9,2493.36,8447.36,181.90,82.08,165.49,8.32,8.09,62321.19,5689.38,4807.84,32.52,0.43,0.98,0.63,2.39,-0.32
std,3931.94,2910.76,9042.3,3063.53,15296.94,232.79,88.94,205.50,17.38,16.75,32358.81,13258.95,2252.22,24.14,0.50,0.12,0.18,2.29,0.27
min,4.00,0.00,0.0,0.00,5.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,-0.91
25%,917.00,259.00,730.0,806.00,2231.00,73.00,35.00,67.00,2.00,1.00,39971.00,1248.92,3371.00,13.45,0.00,1.00,0.51,1.00,-0.50
50%,1658.00,706.00,2027.0,1645.00,4553.00,122.00,59.00,112.00,4.00,4.00,55000.00,2707.28,4551.00,25.75,0.00,1.00,0.64,2.00,-0.33
75%,3076.00,1565.00,4678.0,3093.00,9175.00,212.00,101.00,195.00,9.00,9.00,77691.00,5018.28,5922.00,46.13,1.00,1.00,0.77,3.00,-0.20
max,97861.00,110028.00,370254.0,75320.00,566395.00,5897.00,1944.00,5152.00,661.00,709.00,250001.00,205500.00,53812.00,100.00,1.00,1.00,1.00,42.00,6.00


In [48]:
round(merger_sample[tract_comparison_list].describe(), 2)

,vol_100k,vol_250k,vol_1mil,vol_sbus,sbl_total,sbl_total_num,num_sbus,num_100k,num_250k,num_1mil,medincome,popden,poptot,pminority,principal_city_binary,urban_rural_binary,pcollege,total_branches_tract,avg_branch_growth_tract
count,91951.00,91951.00,91951.00,91951.00,91951.00,91951.00,91951.00,91951.00,91951.00,91951.00,91951.00,91946.00,91951.00,91951.00,76189.00,78293.00,91851.00,91951.00,79863.00
mean,1809.06,1039.49,3226.88,1904.87,6075.43,125.01,57.13,112.92,6.06,6.03,57674.38,3519.03,4658.01,26.63,0.34,0.96,0.60,2.40,0.05
std,2411.64,1894.76,5972.23,2214.30,9806.16,146.10,58.81,128.78,11.32,10.99,28244.29,5873.20,2064.65,23.23,0.47,0.21,0.17,1.93,0.35
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,-0.91
25%,645.00,203.00,500.00,583.00,1679.00,51.00,23.00,47.00,1.00,1.00,38349.00,802.00,3289.50,8.96,0.00,1.00,0.46,1.00,0.00
50%,1188.00,569.00,1605.00,1273.00,3499.00,88.00,41.00,80.00,3.00,3.00,51414.00,2173.21,4430.00,18.93,0.00,1.00,0.59,2.00,0.00
75%,2161.00,1218.00,3716.50,2444.00,7005.00,151.00,72.00,137.00,7.00,7.00,70777.50,4119.26,5716.00,37.02,1.00,1.00,0.73,3.00,0.00
max,107825.00,110028.00,370254.00,75320.00,566395.00,5897.00,1944.00,5152.00,661.00,709.00,250001.00,161916.67,38169.00,100.00,1.00,1.00,1.00,44.00,8.00
